In [1]:
import pandas as pd
import random

import re
import tensorflow as tf
import numpy as np
#from sklearn import metrics, cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.contrib import layers
from tensorflow.contrib import learn


In [2]:
test_orig = pd.read_csv('E:\Machine Learning\Competitions - Kaggle\Titanic Machine Learning from Disaster\Data/test.csv')
train_orig = pd.read_csv('E:\Machine Learning\Competitions - Kaggle\Titanic Machine Learning from Disaster\Data/train.csv')
train = train_orig.copy()
test = test_orig.copy()



In [3]:
def isNaN(x):
    return x != x
#fileds present in the data 
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
#Lets focus on converting all the column data to be numaric, We can see if missing data is going to cause problems after the first iteration is done
#easy to convert text data 
for key in train.columns:
    print("Number of Classes in ", key, " = ", len(train[key].unique()), 
          " Number of NAN = " ,sum(isNaN(train[key])))

Number of Classes in  PassengerId  =  891  Number of NAN =  0
Number of Classes in  Survived  =  2  Number of NAN =  0
Number of Classes in  Pclass  =  3  Number of NAN =  0
Number of Classes in  Name  =  891  Number of NAN =  0
Number of Classes in  Sex  =  2  Number of NAN =  0
Number of Classes in  Age  =  89  Number of NAN =  177
Number of Classes in  SibSp  =  7  Number of NAN =  0
Number of Classes in  Parch  =  7  Number of NAN =  0
Number of Classes in  Ticket  =  681  Number of NAN =  0
Number of Classes in  Fare  =  248  Number of NAN =  0
Number of Classes in  Cabin  =  148  Number of NAN =  687
Number of Classes in  Embarked  =  4  Number of NAN =  2


In [5]:
# Age has  177 missing
# Cabin has 687 missing
# Embarked 2 missing
from sklearn.preprocessing import LabelEncoder

In [6]:
#Replace Missing Embarked value 
#label encode sex and embarked
for k in ['Embarked', 'Sex']:
    for tr2 in [train, test]:
        tr2[k] = tr2[k].fillna('N')
        le = LabelEncoder().fit(tr2[k])
        tr2[k] = le.transform(tr2[k])

In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3


In [8]:
def getPName(st):
    if isNaN(st):
        return st
    dt = st.split(',')
    lastName = dt[0]
    fullName = dt[1].split(' ')
    fullName = list(filter(lambda f:f!='', fullName))
    pn = fullName[0]
    if pn == 'Mlle.':
        pn = 'Miss.'
    elif pn == 'Mme.':
        pn = 'Mrs.'
    elif pn == 'Ms.':
        pn = 'Miss.'
    return pn
def getLName(st):
    if isNaN(st):
        return st
    dt = st.split(',')
    lastName = dt[0]
    return lastName

In [9]:
titleMap = {}
cnt =0
for tr2 in [train, test]:
    titles = tr2.apply(lambda r:getPName(r['Name']), axis=1).unique()
    for idx in range(0, len(titles)):
        if not titles[idx] in titleMap:
            titleMap[titles[idx]] = cnt
            cnt+=1
for tr2 in [train, test]:
    tr2['Titles'] = tr2.apply(lambda r:titleMap[getPName(r['Name'])], axis=1)
print(train['Titles'].unique())
print(titleMap)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
{'Rev.': 5, 'the': 12, 'Sir.': 9, 'Mrs.': 1, 'Major.': 7, 'Lady.': 8, 'Jonkheer.': 13, 'Mr.': 0, 'Dr.': 6, 'Dona.': 14, 'Col.': 10, 'Don.': 4, 'Miss.': 2, 'Capt.': 11, 'Master.': 3}


In [10]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titles
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3,0


In [11]:
import numpy as np
lnameMap = {}
cnt = 0
for tr2 in [train, test]:
    lname = tr2.apply(lambda r:getLName(r['Name']), axis=1).unique()
    for idx in range(0, len(lname)):
        if not lname[idx] in lnameMap:
            lnameMap[lname[idx]] = cnt
            cnt +=1
for tr2 in [train, test]:
    tr2['LName'] = tr2.apply(lambda r:lnameMap[getLName(r['Name'])], axis=1)

train['LName_t'] = tf.contrib.layers.embed_sequence(np.asarray(train['LName']), 
                                                  vocab_size=len(train['LName']),
                                                 embed_dim=6)

In [12]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titles,LName,LName_t
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3,0,0,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,1,1,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3,2,2,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3,1,3,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3,0,4,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."


In [13]:
# Age has lot of missing data  (177 entries)
# Let's try to fill them based on average data based on different titles 
titleMapAvgAge = {}
for k in titleMap:
    num_valid = sum((train['Titles'] == titleMap[k]) & ~isNaN(train['Age'] ))
    num_nan = sum((train['Titles'] == titleMap[k]) & isNaN(train['Age'] ))
    sum_age   = sum(train.loc[(train['Titles'] == titleMap[k]) & ~isNaN(train['Age']),'Age'])
    if (num_valid != 0):
        avg = sum_age/num_valid
    else:
        avg = 0
    titleMapAvgAge[titleMap[k]] = avg
    print('title=',k, ' NumberOfItems:',num_valid,'/',num_nan, ' Average Age:',avg)
# Plugin all the missing ages based on title 

title= Rev.  NumberOfItems: 6 / 0  Average Age: 43.1666666667
title= the  NumberOfItems: 1 / 0  Average Age: 33.0
title= Sir.  NumberOfItems: 1 / 0  Average Age: 49.0
title= Mrs.  NumberOfItems: 109 / 17  Average Age: 35.7889908257
title= Major.  NumberOfItems: 2 / 0  Average Age: 48.5
title= Lady.  NumberOfItems: 1 / 0  Average Age: 48.0
title= Jonkheer.  NumberOfItems: 1 / 0  Average Age: 38.0
title= Mr.  NumberOfItems: 398 / 119  Average Age: 32.3680904523
title= Dr.  NumberOfItems: 6 / 1  Average Age: 42.0
title= Dona.  NumberOfItems: 0 / 0  Average Age: 0
title= Col.  NumberOfItems: 2 / 0  Average Age: 58.0
title= Don.  NumberOfItems: 1 / 0  Average Age: 40.0
title= Miss.  NumberOfItems: 149 / 36  Average Age: 21.8456375839
title= Capt.  NumberOfItems: 1 / 0  Average Age: 70.0
title= Master.  NumberOfItems: 36 / 4  Average Age: 4.57416666667


In [14]:
# Plugin all the missing ages based on title 

def plugAge(r):
    if not isNaN(r['Age']):
        return r['Age']
    return titleMapAvgAge[r['Titles']]
train['Age'] = train.apply(lambda r:plugAge(r), axis=1)
test['Age'] = test.apply(lambda r:plugAge(r), axis=1)

In [15]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titles,LName,LName_t
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3,0,0,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,1,1,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3,2,2,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3,1,3,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3,0,4,"Tensor(""EmbedSequence/embedding_lookup:0"", sha..."


In [16]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Titles', 'LName',
       'LName_t'],
      dtype='object')

In [17]:
featureListKey =[ 'Embarked', 'Sex', 'SibSp', 'Parch', 'Titles', 'LName', 'Age', 'Fare', 
                 'Pclass'  
            ]

In [18]:
fullDt = pd.concat([train, test])
X = train[featureListKey].copy()
y = train['Survived'].copy()

#normalize Age
for key in ['Age', 'Fare']:
    X[key]=((X[key] - X[key].mean())/(X[key].max() - X[key].min()))
    
X['Pclass'] = X['Pclass'] - 1

XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# Tensor flow feature setup 
f_embarked = tf.feature_column.numeric_column(key='Embarked')
f_sex = tf.feature_column.numeric_column(key='Sex')
f_sibsp = tf.feature_column.numeric_column(key='SibSp')
f_parch = tf.feature_column.numeric_column(key='Parch')
f_age = tf.feature_column.numeric_column(key='Age')
f_titles = tf.feature_column.numeric_column(key='Titles')
f_fare = tf.feature_column.numeric_column(key='Fare')
f_pclass = tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key='Pclass',num_buckets=3), 3)
#f_titles = tf.feature_column.embedding_column(
#    tf.feature_column.categorical_column_with_hash_bucket(key='Titles',hash_bucket_size=20), 10)
f_columns= [ f_embarked, f_sex, f_sibsp, f_parch, f_pclass, f_age, f_titles]
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=XTrain, 
                                                y=yTrain,
                                                batch_size=150,
                                                num_epochs=None,
                                                shuffle=True)
test_input_fn = tf.estimator.inputs.pandas_input_fn(x=XTest, 
                                                y=yTest,
                                                num_epochs=1,
                                                shuffle=False)
starter_learning_rate = 0.02
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)
my_nn = tf.estimator.DNNClassifier(feature_columns=f_columns,
                                  hidden_units=[40,40],
                                  activation_fn = tf.nn.relu,
                                  dropout=0.1,
                                  n_classes=2,
                                  optimizer=tf.train.FtrlOptimizer(learning_rate=0.02, 
                                           l2_regularization_strength=0.2))

In [53]:
my_nn.train(input_fn=train_input_fn, steps=1000 )
ev=my_nn.evaluate(input_fn=test_input_fn)
print("Loss:%s" %ev["loss"], " Accuracy:", ev['accuracy'])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:loss = 114.998, step = 1
INFO:tensorflow:global_step/sec: 220.642
INFO:tensorflow:loss = 65.4727, step = 101 (0.469 sec)
INFO:tensorflow:global_step/sec: 228.517
INFO:tensorflow:loss = 66.9275, step = 201 (0.453 sec)
INFO:tensorflow:global_step/sec: 213.368
INFO:tensorflow:loss = 56.8769, step = 301 (0.453 sec)
INFO:tensorflow:global_step/sec: 246.039
INFO:tensorflow:loss = 71.3885, step = 401 (0.406 sec)
INFO:tensorflow:global_step/sec: 213.46
INFO:tensorflow:loss = 88.4001, step = 501 (0.468 sec)
INFO:tensorflow:global_step/sec: 226.902
INFO:tensorflow:loss = 62.2971, step = 601 (0.445 sec)
INFO:tensorflow:global_step/sec: 235.54
INFO:tensorflow:loss = 82.5706, step = 701 (0.420 sec)
INFO:tensorflow:global_step/sec: 240.04
INFO:tensorflow:loss = 53.8905, step = 801 (0.417 sec)
INFO:tensorflow:global_step/sec: 228.427
INFO:tensorflow:loss = 56.8013, step = 901 (0.438 sec)
INFO:tensorflow:Loss for final step: 57.8637.
INFO:ten

In [21]:
XEval=test[featureListKey].copy()
for key in ['Age', 'Fare']:
    XEval[key]=((XEval[key] - XEval[key].mean())/(XEval[key].max() - XEval[key].min()))
XEval['Pclass'] = XEval['Pclass'] - 1

In [22]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(x=XEval, 
                                                num_epochs=1,
                                                shuffle=False)
pred = my_nn.predict(input_fn=pred_input_fn)

In [23]:
v=[]
for i, p in enumerate(pred):
    v.append(p['class_ids'][0])

In [25]:
result = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':v})
result.to_csv('E:\Machine Learning\Competitions - Kaggle\Titanic Machine Learning from Disaster\submission/result.csv', index=False)